In [1]:
import cv2
import os
import sys
import numpy as np
from pathlib import Path

#creates a figure with 10 (width) x 5 (height) inches
np.set_printoptions(threshold=sys.maxsize)

directory='/home/mostafa/Documents/OCR/'
index_you_want = -1#-1 for all
index_currently = 1
enable_img_show=False

for filename in os.listdir(directory):
    print(filename)
    if index_you_want!=-1:
        if index_you_want!=index_currently:
            index_currently+=1
            continue
        index_currently+=1
    
    if filename.endswith(".jpg") or  filename.endswith(".png"):
        f=os.path.join(directory, filename)
        print(f)
        # Read the original image
        img = cv2.imread(f)
        #img = cv2.resize(img, (320,320), interpolation = cv2.INTER_AREA)
        
        # Display original image
        if enable_img_show:
            #cv2.imshow('Original', img)
            cv2.imshow('Image',img)
            cv2.waitKey(0)

        # Convert to graycsale
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if enable_img_show:
            #cv2.imshow('img_gray', img_gray)
            cv2.imshow('Image',img_gray)
            cv2.waitKey(0)

        # Blur the image for better edge detection
        #img_blur = cv2.GaussianBlur(img_gray, (1,1), 0)
        img_blur=img_gray
        if enable_img_show:
            #cv2.imshow('img_gray', img_gray)
            cv2.imshow('Image',img_blur)
            cv2.waitKey(0)
        #cv2.imshow('img_blur', img_blur)
        #cv2.waitKey(0)

        # Sobel Edge Detection
        sobelx = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=5) # Sobel Edge Detection on the X axis
        #cv2.imshow('Sobel X', sobelx)
        #cv2.waitKey(0)

        sobely = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=5) # Sobel Edge Detection on the Y axis
        #cv2.imshow('Sobel Y', sobely)
        #cv2.waitKey(0)

        sobelxy = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=5) # Combined X and Y Sobel Edge Detection
        #cv2.imshow('Sobel X Y using Sobel() function', sobelxy)
        #cv2.waitKey(0)

        # Canny Edge Detection
        edges = cv2.Canny(image=img_blur, threshold1=100, threshold2=200) # Canny Edge Detection
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (1, 1))
        edges = cv2.dilate(edges, kernel)
    
        # Display Canny Edge Detection Image
        if enable_img_show:
            #cv2.imshow('Canny Edge Detection', edges)
            cv2.imshow('Image',edges)
            cv2.waitKey(0)
        
        #connected componenet #connected componenet #connected componenet #connected componenet #connected componenet
        
        (numLabels, labels, stats, centroids) = cv2.connectedComponentsWithStats(edges, 4 , cv2.CV_32S)
        #labels_colored = cv2.normalize(labels, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        
        #labels_colored = cv2.cvtColor(labels.astype("uint8"),cv2.COLOR_GRAY2BGR)
        #labels_colored = (labels_colored<-1).astype("uint8")
        #for i in range(0, numLabels):
        #   componentMask = (labels == i).astype("uint8") * 255
#            labels_colored[:,:,i%3] += componentMask-i
#            if enable_img_show:
#                print('labels_colored '+str(i))
#                cv2.imshow('labels_colored', labels_colored)
#                cv2.waitKey(0)
                
        areas = stats[:, cv2.CC_STAT_AREA]
        
        print('numLabels=',numLabels)
        print("len areas=",len(areas)," areas.shape=",areas.shape)
        print("labels.shape",labels.shape)
        
        #output = img.copy()
        output = cv2.cvtColor(img_blur.copy(), cv2.COLOR_GRAY2BGR)
        
        if enable_img_show:
                #cv2.imshow('output', output)
                cv2.imshow('Image',output)
                cv2.waitKey(0)
        
        img_width =  min(img.shape[0],img.shape[1])
        character_low_limit_area=(img_width/200)*(img_width/200)
        character_high_limit_area=(img_width/10)*(img_width/10)
        needed = sum(map(lambda x : x>character_low_limit_area and x<character_high_limit_area, stats[:,cv2.CC_STAT_AREA]))

        print("needed=",needed," numLabels=",numLabels)

        needed_counter=0
        
        # loop over the number of unique connected component labels######################Looooop
        for i in range(0, numLabels):
            
            #enable_img_show = i%15==0
            
            x = stats[i, cv2.CC_STAT_LEFT]
            y = stats[i, cv2.CC_STAT_TOP]
            w = stats[i, cv2.CC_STAT_WIDTH]
            h = stats[i, cv2.CC_STAT_HEIGHT]
            
            LENGTH=areas[i]
            AREA=w*h
            RATIO = LENGTH/AREA
            float_id = i/numLabels
            
            if RATIO<0.03:
                print('Processing ',i,' Discard due to RATIO=',RATIO)
                continue
                
            if not (areas[i]>character_low_limit_area and areas[i]<character_high_limit_area):
                #print('Processing ',i,' Discard:',areas[i])
                continue
            
            #print('Processing ',i)
            #print("->",needed_counter,"/",needed,"/area=",stats[i, cv2.CC_STAT_AREA])

            # construct a mask for the current connected component by
            # finding a pixels in the labels array that have the current
            # connected component ID

            labels_i = (labels == i).astype("uint8")
            #length = np.sum(sum(labels_i)) #this equals area[i]
            
            componentMask = labels_i * 255
            
            contours, hierarchy  = cv2.findContours(componentMask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            #print(contours)
            cv2.drawContours(output, contours, -1, (255,255,255), 5)
            
            #componentMask = cv2.cvtColor(componentMask,cv2.COLOR_GRAY2BGR)
            
            edges_comononent = cv2.cvtColor(edges,cv2.COLOR_GRAY2BGR)
            edges_comononent[:,:,1]=componentMask
            
            cv2.putText(edges_comononent, str(needed_counter) , (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(edges_comononent, "length="+str(LENGTH)+", area="+str(AREA)+" rat: "+format(RATIO,".2f") , (30,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            output[:,:,1] |= (labels_i * int(255*RATIO))
            
            if enable_img_show:
                cv2.imshow('componentMask', edges_comononent)
            #cv2.rectangle(output, (x, y),((x + w),(y + h)),(255, 255, 255), -1)
            #cv2.circle(output, (int(centroids[i][0]), int(centroids[i][1])), 1, (0, 255, 255), 5)    
            
            #output[:,:,2]+= (labels_i * int(255*RATIO))
            #output[:,:,1]+= (labels_i * int(255*RATIO))
            #output[:,:,0]+= (labels_i * int(255*RATIO))
            
            #cv2.putText(output, str(needed_counter) , (x,y+h), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            # show our output image and connected component mask
            #output = cv2.resize(output, (640,640), interpolation = cv2.INTER_AREA)
            
            cv2.rectangle(componentMask, (30, 20),(150,20),(255, 255, 255), -1)
            cv2.putText(output, 'output:'+str(i) , (30,20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, int(255*float_id),int((1-float_id)*255)), 2, cv2.LINE_AA)
            
            if enable_img_show:
                #cv2.imshow('output', output)
                cv2.imshow('Image',output)
                cv2.waitKey(0)

            #https://stackoverflow.com/questions/678236/how-to-get-the-filename-without-the-extension-from-a-path-in-python
            cv2.imwrite('/home/mostafa/Documents/OCR/compare_original_with_processed/{}{}.jpg'.format(Path(filename).stem,'_original'), img)
            cv2.imwrite('/home/mostafa/Documents/OCR/compare_original_with_processed/{}{}.jpg'.format(Path(filename).stem,'_processed'), output)

            needed_counter=needed_counter+1
        
        #output now is filled with whites
        output=output>250
        output=output.astype("uint8") *255
        print("output.shape=",output.shape)
        output=255-img*output
        cv2.imwrite('/home/mostafa/Documents/OCR/compare_original_with_processed/{}{}.jpg'.format(Path(filename).stem,'_final'), output)
        
        output = cv2.resize(output, (512,512), interpolation = cv2.INTER_AREA)
        if enable_img_show:
            #cv2.imshow('output', output)
            cv2.imshow('Image',output)
            cv2.waitKey(0)
            cv2.destroyAllWindows()  

book_cover_perfect2.jpg
/home/mostafa/Documents/OCR/book_cover_perfect2.jpg
numLabels= 584
len areas= 584  areas.shape= (584,)
labels.shape (1000, 1100)
needed= 306  numLabels= 584
Processing  1  Discard due to RATIO= 0.006375306583214671
Processing  90  Discard due to RATIO= 0.010115456625754108
Processing  121  Discard due to RATIO= 0.018956419777213213
Processing  153  Discard due to RATIO= 0.021951528993782516
Processing  335  Discard due to RATIO= 0.011017252167694646
Processing  347  Discard due to RATIO= 0.02680306905370844
output.shape= (1000, 1100, 3)
book_cover_arabic_3d.jpg
/home/mostafa/Documents/OCR/book_cover_arabic_3d.jpg
numLabels= 279
len areas= 279  areas.shape= (279,)
labels.shape (600, 600)
needed= 98  numLabels= 279
Processing  1  Discard due to RATIO= 0.02950778012354367
Processing  211  Discard due to RATIO= 0.024196460816179127
Processing  219  Discard due to RATIO= 0.024283559577677225
output.shape= (600, 600, 3)
face
mix_color_hard.jpg
/home/mostafa/Documents/